In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

## Define our coupled derivatives to integrate

In [2]:
def dydx(x,y):
    #Set derivatives
    #our equation is d^2y/dx^2=-y
    #sp dydx=z
    #dzdx=-y
    #set y=y[0]
    #set x=y[1]
    #declare an array
    y_derivs=np.zeros(2)
    
    #set dydx=z
    y_derivs[0]=y[1]
    #set dzdx=-y
    y_derivs[1]=-1*y[0]
    #here we have to return an array
    return y_derivs

## Define the 4th order RK method

In [5]:
def rk4_mv_core(dydx,xi,yi,nv,h):
    #declare k? arrays
    k1=np.zeros(nv)
    k2=np.zeros(nv)
    k3=np.zeros(nv)
    k4=np.zeros(nv)
    
    #define x at 1/2 step
    x_ipoh=xi+.5*h
    
    #define x at 1 step
    x_ipo=xi+h
    
    #declare a temp y array
    y_temp=np.zeros(nv)
    
    #get k1 values 
    y_derivs=dydx(xi,yi)
    k1[:]=h*y_derivs[:]
    
    #get k2 values
    y_temp[:]=yi[i]+.5*k1[:]
    y_derivs=dydx(x_ipoh,y_temp)
    k2[:]=h*y_derivs[:]
    
    #get k3 values
    y_temp[:]=yi[:]+.5*k1[:]
    y_derivs=dydx(x_ipoh,y_temp)
    k2[:]=h*y_derivs[:]
    
    #get k4 values
    y_temp[:]=yi[:]+k3[:]
    y_derivs=dydx(x_ipo,y_temp)
    k4[:]=h*y_derivs[:]
    
    #advance y by a step h
    yipo=yi+(k1+2*k2+2*k3+k4)/6
    return yipo

In [8]:
def rk4_mv_ad(dydx,x_i,y_i,nv,h,tol):
    
    #define safety scale
    SAFETY=.9
    H_NEW_FAC=2.0
    
    #set a maximum number of iterations
    imax=10000
    
    i=0
    #create an error
    Delta=np.full(nv,2*tol)
    
    #remember the step
    h_step=h
    
    #adjust step
    while(Delta.max()/tol>1.0):
        #estimate our error by taking one step of size h v. two steps of size h/2
        y_2=rk4_mv_core(dydx,x_i,y_i,nv,h_step)
        y_1=rk4_mv_core(dydx,x_i,y_i,nv,.5*h_step)
        y_11=rk4_mv_core(dydx,x_i+.5*h_step,y_1,nv,.5*h_step)
        
        #compute an error
        Delta=np.fabs(y_2-y_1)
        
        #if the error is too large, take a smaller step
        if (Delta.max()>tol):
            #our error is too large, take a smaller step
            h_step*=SAFETY*(Delta.max()/tol**(-.25))
            
        #check iteration
        if(i>=imax):
            raise StopIteration("Ending after i=",i)
        
        i+=1
    #next time, try to take a bigger step
    h_new=np.fmin(h_step*Delta.max()/tol**(-.9),h_step*H_NEW_FAC)
    
    return y_2,h_new,h_step

In [10]:
def rk4_mv(dfdx, a,b,y_a,tol):
    #define starting step
    xi=a
    yi=y_a.copy()
    
    #an initial step size==make very small
    h=1.0e-4*(b-a)
    imax=10000
    i=0
    
    nv=len(y_a)
    #set initial conditions
    x=np.full(1,a)
    y=np.full((1,nv),y_a)
    
    flag=1
    
    while(flag):
        yi_new,h_new,h_step=rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        h=h_new
        
        #prevent an overshoot
        h=b-xi
        
        #recalculate y_i+1
        yi_new,h_new,h_step=rk4_mv_ad(dydx,xi,yi,nv,h,tol)
        
        flag=0
        
    #update values
    xi+= h_step
    yi[:]=yi_new[:]
    
    #add the step to the arrays
    